# I-129 Release/Transport from the Repository

### This notebook computes I-129 release from the repository and transport in the geosphere

In [3]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as tk 
import numpy as np

# Build Functions

### I-129 Mass Calculation
This section extracts the concentration of I-129 in the spent fuel from the "cooling_reactor.csv" files. With the parameters of the canister, it is possible to calculate the quantity of I-129 in each canister.

In [4]:
def get_I129_mass(workspace_path, reactor, reactor_parameters):
    '''
    Return the molar mass of I129 in the reactor-specific canister (mol/canister)
    '''
    ## ---- Parameters:
    # workspace_path (str) - path to R2R4SNF directory
    # reactor (str) - reactor type
    # reactor_parameters (dict) - reactor parameter dictionary
    ## ---- Returns:
    # I129_mass - I-129 mass per canister

    # Cooling reactor SNF metrics from OpenMC simulation
    cooling_result = pd.read_csv(workspace_path + '/results/' + reactor +'/reactor_simulation/summary/cooling_' + reactor + '.csv')
    
    # I129 parameters 
    half_life  = 15.7 * 365 * 24 * 60 * 60 * (10**6) # in seconds
    Na         = 6.022 * (10**23) # Avogadro
    molar_mass = 129 # g/mol

    # Calculate heavy metal mass and proportion of I-129 per canister, then multiply to get I-129 mass
    canister_mass = float(reactor_parameters['hm_per_asm']) * float(reactor_parameters['# fuel units']) 
    I129_prop        = ((cooling_result['I129 (atom/cm3)'][0] / Na) * cooling_result['Volume Ueq'][0]) / cooling_result['Mass U'][0]
    I129_mass        = I129_prop * canister_mass
    
    return I129_mass

### Fuel Matrix Dissolution Calculation

The degradation rate of the matrix in the environment will depend on the characteristics of the matrix. In this study, it is supposed that the degradation rate of the matrix is constant. The matrix contained in one canister is modelled by a sphere of equivalent mass: this geometrical trasnformation enables to easily consider the evolution of the matrix mass and volume during the degradation process. The radius of the sphere is given by:

$r_{0}=\frac{3M}{4\rho\pi}^{1/3}$

Then, considering a constant degradation rate of the matrix in the environment, the evolution of the radius with time is given as: $ r(t)=r_{0} - R*t$ with $R$ the degradation rate of the matrix. Then, the fraction of the matrix remaining in the canister is simply obtained with: $ f(t)= (1-\frac {Rt}{r_{0}\rho})^3 $

In [5]:
def get_matrix_remaining(reactor_parameters, level, t):
    '''
    Calculate the fraction of the matrix remaining in the canister at time t in years
    '''
    ## ---- Parameters:
    # reactor_parameters (dict) - reactor parameter dictionary
    # level (str) - degradation rate (low or high)
    # t (list) - exponential time series for measuring waste over time
    ## ---- Returns:
    # fraction (list) - fraction of the matrix remaining in the canister at time t in years   
   
    # Canister and matrix parameters
    tot_mass = float(reactor_parameters['mass_per_fuel_unit']) * float(reactor_parameters['# fuel units'])
    density = float(reactor_parameters['density'])
    deg_rate = float(reactor_parameters[level + '_degradation_rate'])

    # Calculate equivalent radius for a matrix sphere with the same mass using formula above
    radius = ((3 * tot_mass) / (4 * np.pi * density))**(1/3) 
    
    fraction = (1 - deg_rate / (density * radius) * (10**-4) * t * 365)**3
    fraction[fraction < 0] = 0
    return fraction

### I-129 Canister Release Calculation

The conservative behaviour of I129 in the environment enables to make several hypothesis:
- The concentration of I129 in the spent fuel is constant (no decay)
- I129 is released instantly at canister failure $(2,5%)$ and the rest is released congruently to the waste matrix, at the same rate.

Therefore, the total I129 released in the environment after $t$ is given as: $M_{I129}(t)=M_{I129}(0)(IRF+(1-IRF)(1-(1-\frac {Rt}{r_{0}\rho})^3))$

In [6]:
def get_I129_release_can(workspace_path, reactor, reactor_parameters, level, t):
    """
    Calculate the total quantity of I129 (in mol) released in the environment at time t in years for one canister
    """
    ## ---- Parameters:
    # workspace_path (str) - path to R2R4SNF directory
    # reactor (str) - reactor type
    # reactor_parameters (dict) - reactor parameter dictionary
    # level (str) - degradation rate (low or high)
    # t (list) - exponential time series for measuring waste over time

    ## ---- Returns:
    # I129_release (df) - I129 (in mol) released in the environment at time t in years for one canister   

    # fraction of the matrix remaining in the waste package
    IRF = float(reactor_parameters['IRF'])
    matrix_f = get_matrix_remaining(reactor_parameters, level, t) 
    I129_mass = get_I129_mass(workspace_path, reactor, reactor_parameters) 
        
    I129_release = I129_mass * (IRF + (1 - IRF) * (1 - matrix_f))
    return I129_release

In [7]:
def build_dfs(workspace_path, reactors, parameters, level, t):
    """
    Save dataframes with the values of I129 released in the environment for all reactor types
    """
    ## ---- Parameters:
    # workspace_path (str) - path to R2R4SNF directory
    # reactors (list) - reactor types
    # reactor_parameters (dict) - reactor parameter dictionary
    # level (str) - degradation rate (low or high)
    # t (list) - exponential time series for measuring waste over time
    ## ---- Returns:
    # remaining_waste (df) - remaining waste fraction over time for all reactor types
    # I129_release (df) - I129 release over time for all reactor types
    
    # Build data frames with fraction of waste and I-129 release for each reactor over time
    remaining_waste = pd.DataFrame()
    I129_release = pd.DataFrame()
    
    for reactor in reactors:
        # Calculate fraction of waste remaining over time
        remaining_waste[reactor] = get_matrix_remaining(parameters[reactor], level, t)
        
        # Calculate moles of I-129 released over time
        I129_release[reactor] = get_I129_release_can(workspace_path, reactor, parameters[reactor], level, t)

    # Scale to GWe.y and the Yucca Mountain limit (70,000MTHM)
    remaining_waste.index = t
    I129_release.index = t

    return (remaining_waste, I129_release)

# PFLOTRAN I/O Functions
This part calculates the total quantity of I129 released in the environment with the paramaters with the same methodology as presented above from the study P.E Mariner, Application of generic Disposal System Models, 2015. 

In [8]:
def I129_release_rate4PFLOTRAN(workspace_path, I129_release):
    """ 
    Saves I129_release data to create a time-varying concentration in a flux at the source term location for PFLOTRAN
    """
    ## ---- Parameters:
    # workspace_path (str) - path to R2R4SNF directory
    # I129_release (df) - I129 release over time for all reactor types
    
    for reactor in I129_release.columns:
        
        time = I129_release.index
        
        r = I129_release[reactor]
        t = np.diff(time)
        r = np.diff(r)

        time = time[1:]
        rate = r/t
        time = time[::2]
        rate = rate[::2]
        cons = rate/1e-6

        with open(workspace_path + 'inputs/PFLOTRAN/pflotran_'+reactor+'/CONSTRAINT_reactor.txt','w') as file:
            for i in range(len(time)):
                file.write('CONSTRAINT cons'+str(i)+'\n')
                file.write('  CONCENTRATIONS \n')
                file.write('    I129   {cons:.2e}   T\n'.format(cons = cons[i]))
                file.write('  /\n')
                file.write('END\n')
        file.close()
    
        with open(workspace_path + 'inputs/PFLOTRAN/CONSTRAINT_LIST.txt','w') as file:
            file.write('0.d0 initial \n')
            for i in range(len(time)):
                file.write('{time:.2e}'.format(time=time[i])+'  cons'+str(i)+'\n')
        file.close()

In [9]:
def df_I129_PFLOTRAN2cons(workspace_path, reactors):
    """
    Reads the PFLOTRAN output file at the well loation
    """
    ## ---- Parameters:
    # workspace_path (str) - path to R2R4SNF directory
    # reactors (list) - reactor types
    ## ---- Returns:
    # df (df) - PFLOTRAN output file
    
    half_life  = 15.7*365*24*60*60*(10**6) # in seconds
    lam = np.log(2)/half_life
    
    df=pd.DataFrame()
    for reactor in reactors:
        result=pd.read_csv(workspace_path + 'inputs/PFLOTRAN/pflotran_'+reactor+'/pflotran-obs-0.pft',
                           delimiter=r"\s+", header = 1, 
                           names = ["Time [y]", "Liquid Pressure [Pa] well (750) (1.4990E+04 10. 0.5)","Liquid Saturation well (750) (1.4990E+04 10. 0.5)","Free I129 [M] well (750) (1.4990E+04 10. 0.5)","Liquid Pressure [Pa] well (751) (1.5010E+04 10. 0.5)","Liquid Saturation well (751) (1.5010E+04 10. 0.5)","Free I129 [M] well (751) (1.5010E+04 10. 0.5)","Liquid Pressure [Pa] well (1750) (1.4990E+04 10. 1.5)","Liquid Saturation well (1750) (1.4990E+04 10. 1.5)","Free I129 [M] well (1750) (1.4990E+04 10. 1.5)","Liquid Pressure [Pa] well (1751) (1.5010E+04 10. 1.5)","Liquid Saturation well (1751) (1.5010E+04 10. 1.5)","Free I129 [M] well (1751) (1.5010E+04 10. 1.5)","Liquid Pressure [Pa] well (2750) (1.4990E+04 10. 2.5)","Liquid Saturation well (2750) (1.4990E+04 10. 2.5)","Free I129 [M] well (2750) (1.4990E+04 10. 2.5)","Liquid Pressure [Pa] well (2751) (1.5010E+04 10. 2.5)","Liquid Saturation well (2751) (1.5010E+04 10. 2.5)","Free I129 [M] well (2751) (1.5010E+04 10. 2.5)","Liquid Pressure [Pa] well (3750) (1.4990E+04 10. 3.5)","Liquid Saturation well (3750) (1.4990E+04 10. 3.5)","Free I129 [M] well (3750) (1.4990E+04 10. 3.5)","Liquid Pressure [Pa] well (3751) (1.5010E+04 10. 3.5)","Liquid Saturation well (3751) (1.5010E+04 10. 3.5)","Free I129 [M] well (3751) (1.5010E+04 10. 3.5)","Liquid Pressure [Pa] well (4750) (1.4990E+04 10. 4.5)","Liquid Saturation well (4750) (1.4990E+04 10. 4.5)","Free I129 [M] well (4750) (1.4990E+04 10. 4.5)","Liquid Pressure [Pa] well (4751) (1.5010E+04 10. 4.5)","Liquid Saturation well (4751) (1.5010E+04 10. 4.5)","Free I129 [M] well (4751) (1.5010E+04 10. 4.5)","Liquid Pressure [Pa] well (5750) (1.4990E+04 10. 5.5)","Liquid Saturation well (5750) (1.4990E+04 10. 5.5)","Free I129 [M] well (5750) (1.4990E+04 10. 5.5)","Liquid Pressure [Pa] well (5751) (1.5010E+04 10. 5.5)","Liquid Saturation well (5751) (1.5010E+04 10. 5.5)","Free I129 [M] well (5751) (1.5010E+04 10. 5.5)","Liquid Pressure [Pa] well (6750) (1.4990E+04 10. 6.5)","Liquid Saturation well (6750) (1.4990E+04 10. 6.5)","Free I129 [M] well (6750) (1.4990E+04 10. 6.5)","Liquid Pressure [Pa] well (6751) (1.5010E+04 10. 6.5)","Liquid Saturation well (6751) (1.5010E+04 10. 6.5)","Free I129 [M] well (6751) (1.5010E+04 10. 6.5)","Liquid Pressure [Pa] well (7750) (1.4990E+04 10. 7.5)","Liquid Saturation well (7750) (1.4990E+04 10. 7.5)","Free I129 [M] well (7750) (1.4990E+04 10. 7.5)","Liquid Pressure [Pa] well (7751) (1.5010E+04 10. 7.5)","Liquid Saturation well (7751) (1.5010E+04 10. 7.5)","Free I129 [M] well (7751) (1.5010E+04 10. 7.5)","Liquid Pressure [Pa] well (8750) (1.4990E+04 10. 8.5)","Liquid Saturation well (8750) (1.4990E+04 10. 8.5)","Free I129 [M] well (8750) (1.4990E+04 10. 8.5)","Liquid Pressure [Pa] well (8751) (1.5010E+04 10. 8.5)","Liquid Saturation well (8751) (1.5010E+04 10. 8.5)","Free I129 [M] well (8751) (1.5010E+04 10. 8.5)","Liquid Pressure [Pa] well (9750) (1.4990E+04 10. 9.5)","Liquid Saturation well (9750) (1.4990E+04 10. 9.5)","Free I129 [M] well (9750) (1.4990E+04 10. 9.5)","Liquid Pressure [Pa] well (9751) (1.5010E+04 10. 9.5)","Liquid Saturation well (9751) (1.5010E+04 10. 9.5)","Free I129 [M] well (9751) (1.5010E+04 10. 9.5)","Liquid Pressure [Pa] well (10750) (1.4990E+04 10. 10.5)","Liquid Saturation well (10750) (1.4990E+04 10. 10.5)","Free I129 [M] well (10750) (1.4990E+04 10. 10.5)","Liquid Pressure [Pa] well (10751) (1.5010E+04 10. 10.5)","Liquid Saturation well (10751) (1.5010E+04 10. 10.5)","Free I129 [M] well (10751) (1.5010E+04 10. 10.5)","Liquid Pressure [Pa] well (11750) (1.4990E+04 10. 11.5)","Liquid Saturation well (11750) (1.4990E+04 10. 11.5)","Free I129 [M] well (11750) (1.4990E+04 10. 11.5)","Liquid Pressure [Pa] well (11751) (1.5010E+04 10. 11.5)","Liquid Saturation well (11751) (1.5010E+04 10. 11.5)","Free I129 [M] well (11751) (1.5010E+04 10. 11.5)","Liquid Pressure [Pa] well (12750) (1.4990E+04 10. 12.5)","Liquid Saturation well (12750) (1.4990E+04 10. 12.5)","Free I129 [M] well (12750) (1.4990E+04 10. 12.5)","Liquid Pressure [Pa] well (12751) (1.5010E+04 10. 12.5)","Liquid Saturation well (12751) (1.5010E+04 10. 12.5)","Free I129 [M] well (12751) (1.5010E+04 10. 12.5)","Liquid Pressure [Pa] well (13750) (1.4990E+04 10. 13.5)","Liquid Saturation well (13750) (1.4990E+04 10. 13.5)","Free I129 [M] well (13750) (1.4990E+04 10. 13.5)","Liquid Pressure [Pa] well (13751) (1.5010E+04 10. 13.5)","Liquid Saturation well (13751) (1.5010E+04 10. 13.5)","Free I129 [M] well (13751) (1.5010E+04 10. 13.5)","Liquid Pressure [Pa] well (14750) (1.4990E+04 10. 14.5)","Liquid Saturation well (14750) (1.4990E+04 10. 14.5)","Free I129 [M] well (14750) (1.4990E+04 10. 14.5)","Liquid Pressure [Pa] well (14751) (1.5010E+04 10. 14.5)","Liquid Saturation well (14751) (1.5010E+04 10. 14.5)","Free I129 [M] well (14751) (1.5010E+04 10. 14.5)","Liquid Pressure [Pa] well (15750) (1.4990E+04 10. 15.5)","Liquid Saturation well (15750) (1.4990E+04 10. 15.5)","Free I129 [M] well (15750) (1.4990E+04 10. 15.5)","Liquid Pressure [Pa] well (15751) (1.5010E+04 10. 15.5)","Liquid Saturation well (15751) (1.5010E+04 10. 15.5)","Free I129 [M] well (15751) (1.5010E+04 10. 15.5)","Liquid Pressure [Pa] well (16750) (1.4990E+04 10. 16.5)","Liquid Saturation well (16750) (1.4990E+04 10. 16.5)","Free I129 [M] well (16750) (1.4990E+04 10. 16.5)","Liquid Pressure [Pa] well (16751) (1.5010E+04 10. 16.5)","Liquid Saturation well (16751) (1.5010E+04 10. 16.5)","Free I129 [M] well (16751) (1.5010E+04 10. 16.5)","Liquid Pressure [Pa] well (17750) (1.4990E+04 10. 17.5)","Liquid Saturation well (17750) (1.4990E+04 10. 17.5)","Free I129 [M] well (17750) (1.4990E+04 10. 17.5)","Liquid Pressure [Pa] well (17751) (1.5010E+04 10. 17.5)","Liquid Saturation well (17751) (1.5010E+04 10. 17.5)","Free I129 [M] well (17751) (1.5010E+04 10. 17.5)","Liquid Pressure [Pa] well (18750) (1.4990E+04 10. 18.5)","Liquid Saturation well (18750) (1.4990E+04 10. 18.5)","Free I129 [M] well (18750) (1.4990E+04 10. 18.5)","Liquid Pressure [Pa] well (18751) (1.5010E+04 10. 18.5)","Liquid Saturation well (18751) (1.5010E+04 10. 18.5)","Free I129 [M] well (18751) (1.5010E+04 10. 18.5)","Liquid Pressure [Pa] well (19750) (1.4990E+04 10. 19.5)","Liquid Saturation well (19750) (1.4990E+04 10. 19.5)","Free I129 [M] well (19750) (1.4990E+04 10. 19.5)","Liquid Pressure [Pa] well (19751) (1.5010E+04 10. 19.5)","Liquid Saturation well (19751) (1.5010E+04 10. 19.5)","Free I129 [M] well (19751) (1.5010E+04 10. 19.5)","Liquid Pressure [Pa] well (20750) (1.4990E+04 10. 20.5)","Liquid Saturation well (20750) (1.4990E+04 10. 20.5)","Free I129 [M] well (20750) (1.4990E+04 10. 20.5)","Liquid Pressure [Pa] well (20751) (1.5010E+04 10. 20.5)","Liquid Saturation well (20751) (1.5010E+04 10. 20.5)","Free I129 [M] well (20751) (1.5010E+04 10. 20.5)","Liquid Pressure [Pa] well (21750) (1.4990E+04 10. 21.5)","Liquid Saturation well (21750) (1.4990E+04 10. 21.5)","Free I129 [M] well (21750) (1.4990E+04 10. 21.5)","Liquid Pressure [Pa] well (21751) (1.5010E+04 10. 21.5)","Liquid Saturation well (21751) (1.5010E+04 10. 21.5)","Free I129 [M] well (21751) (1.5010E+04 10. 21.5)","Liquid Pressure [Pa] well (22750) (1.4990E+04 10. 22.5)","Liquid Saturation well (22750) (1.4990E+04 10. 22.5)","Free I129 [M] well (22750) (1.4990E+04 10. 22.5)","Liquid Pressure [Pa] well (22751) (1.5010E+04 10. 22.5)","Liquid Saturation well (22751) (1.5010E+04 10. 22.5)","Free I129 [M] well (22751) (1.5010E+04 10. 22.5)","Liquid Pressure [Pa] well (23750) (1.4990E+04 10. 23.5)","Liquid Saturation well (23750) (1.4990E+04 10. 23.5)","Free I129 [M] well (23750) (1.4990E+04 10. 23.5)","Liquid Pressure [Pa] well (23751) (1.5010E+04 10. 23.5)","Liquid Saturation well (23751) (1.5010E+04 10. 23.5)","Free I129 [M] well (23751) (1.5010E+04 10. 23.5)","Liquid Pressure [Pa] well (24750) (1.4990E+04 10. 24.5)","Liquid Saturation well (24750) (1.4990E+04 10. 24.5)","Free I129 [M] well (24750) (1.4990E+04 10. 24.5)","Liquid Pressure [Pa] well (24751) (1.5010E+04 10. 24.5)","Liquid Saturation well (24751) (1.5010E+04 10. 24.5)","Free I129 [M] well (24751) (1.5010E+04 10. 24.5)","Liquid Pressure [Pa] well (25750) (1.4990E+04 10. 25.5)","Liquid Saturation well (25750) (1.4990E+04 10. 25.5)","Free I129 [M] well (25750) (1.4990E+04 10. 25.5)","Liquid Pressure [Pa] well (25751) (1.5010E+04 10. 25.5)","Liquid Saturation well (25751) (1.5010E+04 10. 25.5)","Free I129 [M] well (25751) (1.5010E+04 10. 25.5)","Liquid Pressure [Pa] well (26750) (1.4990E+04 10. 26.5)","Liquid Saturation well (26750) (1.4990E+04 10. 26.5)","Free I129 [M] well (26750) (1.4990E+04 10. 26.5)","Liquid Pressure [Pa] well (26751) (1.5010E+04 10. 26.5)","Liquid Saturation well (26751) (1.5010E+04 10. 26.5)","Free I129 [M] well (26751) (1.5010E+04 10. 26.5)","Liquid Pressure [Pa] well (27750) (1.4990E+04 10. 27.5)","Liquid Saturation well (27750) (1.4990E+04 10. 27.5)","Free I129 [M] well (27750) (1.4990E+04 10. 27.5)","Liquid Pressure [Pa] well (27751) (1.5010E+04 10. 27.5)","Liquid Saturation well (27751) (1.5010E+04 10. 27.5)","Free I129 [M] well (27751) (1.5010E+04 10. 27.5)","Liquid Pressure [Pa] well (28750) (1.4990E+04 10. 28.5)","Liquid Saturation well (28750) (1.4990E+04 10. 28.5)","Free I129 [M] well (28750) (1.4990E+04 10. 28.5)","Liquid Pressure [Pa] well (28751) (1.5010E+04 10. 28.5)","Liquid Saturation well (28751) (1.5010E+04 10. 28.5)","Free I129 [M] well (28751) (1.5010E+04 10. 28.5)","Liquid Pressure [Pa] well (29750) (1.4990E+04 10. 29.5)","Liquid Saturation well (29750) (1.4990E+04 10. 29.5)","Free I129 [M] well (29750) (1.4990E+04 10. 29.5)","Liquid Pressure [Pa] well (29751) (1.5010E+04 10. 29.5)","Liquid Saturation well (29751) (1.5010E+04 10. 29.5)","Free I129 [M] well (29751) (1.5010E+04 10. 29.5)","Liquid Pressure [Pa] well (30750) (1.4990E+04 10. 30.5)","Liquid Saturation well (30750) (1.4990E+04 10. 30.5)","Free I129 [M] well (30750) (1.4990E+04 10. 30.5)","Liquid Pressure [Pa] well (30751) (1.5010E+04 10. 30.5)","Liquid Saturation well (30751) (1.5010E+04 10. 30.5)","Free I129 [M] well (30751) (1.5010E+04 10. 30.5)","Liquid Pressure [Pa] well (31750) (1.4990E+04 10. 31.5)","Liquid Saturation well (31750) (1.4990E+04 10. 31.5)","Free I129 [M] well (31750) (1.4990E+04 10. 31.5)","Liquid Pressure [Pa] well (31751) (1.5010E+04 10. 31.5)","Liquid Saturation well (31751) (1.5010E+04 10. 31.5)","Free I129 [M] well (31751) (1.5010E+04 10. 31.5)","Liquid Pressure [Pa] well (32750) (1.4990E+04 10. 32.5)","Liquid Saturation well (32750) (1.4990E+04 10. 32.5)","Free I129 [M] well (32750) (1.4990E+04 10. 32.5)","Liquid Pressure [Pa] well (32751) (1.5010E+04 10. 32.5)","Liquid Saturation well (32751) (1.5010E+04 10. 32.5)","Free I129 [M] well (32751) (1.5010E+04 10. 32.5)","Liquid Pressure [Pa] well (33750) (1.4990E+04 10. 33.5)","Liquid Saturation well (33750) (1.4990E+04 10. 33.5)","Free I129 [M] well (33750) (1.4990E+04 10. 33.5)","Liquid Pressure [Pa] well (33751) (1.5010E+04 10. 33.5)","Liquid Saturation well (33751) (1.5010E+04 10. 33.5)","Free I129 [M] well (33751) (1.5010E+04 10. 33.5)","Liquid Pressure [Pa] well (34750) (1.4990E+04 10. 34.5)","Liquid Saturation well (34750) (1.4990E+04 10. 34.5)","Free I129 [M] well (34750) (1.4990E+04 10. 34.5)","Liquid Pressure [Pa] well (34751) (1.5010E+04 10. 34.5)","Liquid Saturation well (34751) (1.5010E+04 10. 34.5)","Free I129 [M] well (34751) (1.5010E+04 10. 34.5)","Liquid Pressure [Pa] well (35750) (1.4990E+04 10. 35.5)","Liquid Saturation well (35750) (1.4990E+04 10. 35.5)","Free I129 [M] well (35750) (1.4990E+04 10. 35.5)","Liquid Pressure [Pa] well (35751) (1.5010E+04 10. 35.5)","Liquid Saturation well (35751) (1.5010E+04 10. 35.5)","Free I129 [M] well (35751) (1.5010E+04 10. 35.5)","Liquid Pressure [Pa] well (36750) (1.4990E+04 10. 36.5)","Liquid Saturation well (36750) (1.4990E+04 10. 36.5)","Free I129 [M] well (36750) (1.4990E+04 10. 36.5)","Liquid Pressure [Pa] well (36751) (1.5010E+04 10. 36.5)","Liquid Saturation well (36751) (1.5010E+04 10. 36.5)","Free I129 [M] well (36751) (1.5010E+04 10. 36.5)","Liquid Pressure [Pa] well (37750) (1.4990E+04 10. 37.5)","Liquid Saturation well (37750) (1.4990E+04 10. 37.5)","Free I129 [M] well (37750) (1.4990E+04 10. 37.5)","Liquid Pressure [Pa] well (37751) (1.5010E+04 10. 37.5)","Liquid Saturation well (37751) (1.5010E+04 10. 37.5)","Free I129 [M] well (37751) (1.5010E+04 10. 37.5)","Liquid Pressure [Pa] well (38750) (1.4990E+04 10. 38.5)","Liquid Saturation well (38750) (1.4990E+04 10. 38.5)","Free I129 [M] well (38750) (1.4990E+04 10. 38.5)","Liquid Pressure [Pa] well (38751) (1.5010E+04 10. 38.5)","Liquid Saturation well (38751) (1.5010E+04 10. 38.5)","Free I129 [M] well (38751) (1.5010E+04 10. 38.5)","Liquid Pressure [Pa] well (39750) (1.4990E+04 10. 39.5)","Liquid Saturation well (39750) (1.4990E+04 10. 39.5)","Free I129 [M] well (39750) (1.4990E+04 10. 39.5)","Liquid Pressure [Pa] well (39751) (1.5010E+04 10. 39.5)","Liquid Saturation well (39751) (1.5010E+04 10. 39.5)","Free I129 [M] well (39751) (1.5010E+04 10. 39.5)","Liquid Pressure [Pa] well (40750) (1.4990E+04 10. 40.5)","Liquid Saturation well (40750) (1.4990E+04 10. 40.5)","Free I129 [M] well (40750) (1.4990E+04 10. 40.5)","Liquid Pressure [Pa] well (40751) (1.5010E+04 10. 40.5)","Liquid Saturation well (40751) (1.5010E+04 10. 40.5)","Free I129 [M] well (40751) (1.5010E+04 10. 40.5)","Liquid Pressure [Pa] well (41750) (1.4990E+04 10. 41.5)","Liquid Saturation well (41750) (1.4990E+04 10. 41.5)","Free I129 [M] well (41750) (1.4990E+04 10. 41.5)","Liquid Pressure [Pa] well (41751) (1.5010E+04 10. 41.5)","Liquid Saturation well (41751) (1.5010E+04 10. 41.5)","Free I129 [M] well (41751) (1.5010E+04 10. 41.5)","Liquid Pressure [Pa] well (42750) (1.4990E+04 10. 42.5)","Liquid Saturation well (42750) (1.4990E+04 10. 42.5)","Free I129 [M] well (42750) (1.4990E+04 10. 42.5)","Liquid Pressure [Pa] well (42751) (1.5010E+04 10. 42.5)","Liquid Saturation well (42751) (1.5010E+04 10. 42.5)","Free I129 [M] well (42751) (1.5010E+04 10. 42.5)","Liquid Pressure [Pa] well (43750) (1.4990E+04 10. 43.5)","Liquid Saturation well (43750) (1.4990E+04 10. 43.5)","Free I129 [M] well (43750) (1.4990E+04 10. 43.5)","Liquid Pressure [Pa] well (43751) (1.5010E+04 10. 43.5)","Liquid Saturation well (43751) (1.5010E+04 10. 43.5)","Free I129 [M] well (43751) (1.5010E+04 10. 43.5)","Liquid Pressure [Pa] well (44750) (1.4990E+04 10. 44.5)","Liquid Saturation well (44750) (1.4990E+04 10. 44.5)","Free I129 [M] well (44750) (1.4990E+04 10. 44.5)","Liquid Pressure [Pa] well (44751) (1.5010E+04 10. 44.5)","Liquid Saturation well (44751) (1.5010E+04 10. 44.5)","Free I129 [M] well (44751) (1.5010E+04 10. 44.5)","Liquid Pressure [Pa] well (45750) (1.4990E+04 10. 45.5)","Liquid Saturation well (45750) (1.4990E+04 10. 45.5)","Free I129 [M] well (45750) (1.4990E+04 10. 45.5)","Liquid Pressure [Pa] well (45751) (1.5010E+04 10. 45.5)","Liquid Saturation well (45751) (1.5010E+04 10. 45.5)","Free I129 [M] well (45751) (1.5010E+04 10. 45.5)","Liquid Pressure [Pa] well (46750) (1.4990E+04 10. 46.5)","Liquid Saturation well (46750) (1.4990E+04 10. 46.5)","Free I129 [M] well (46750) (1.4990E+04 10. 46.5)","Liquid Pressure [Pa] well (46751) (1.5010E+04 10. 46.5)","Liquid Saturation well (46751) (1.5010E+04 10. 46.5)","Free I129 [M] well (46751) (1.5010E+04 10. 46.5)","Liquid Pressure [Pa] well (47750) (1.4990E+04 10. 47.5)","Liquid Saturation well (47750) (1.4990E+04 10. 47.5)","Free I129 [M] well (47750) (1.4990E+04 10. 47.5)","Liquid Pressure [Pa] well (47751) (1.5010E+04 10. 47.5)","Liquid Saturation well (47751) (1.5010E+04 10. 47.5)","Free I129 [M] well (47751) (1.5010E+04 10. 47.5)","Liquid Pressure [Pa] well (48750) (1.4990E+04 10. 48.5)","Liquid Saturation well (48750) (1.4990E+04 10. 48.5)","Free I129 [M] well (48750) (1.4990E+04 10. 48.5)","Liquid Pressure [Pa] well (48751) (1.5010E+04 10. 48.5)","Liquid Saturation well (48751) (1.5010E+04 10. 48.5)","Free I129 [M] well (48751) (1.5010E+04 10. 48.5)","Liquid Pressure [Pa] well (49750) (1.4990E+04 10. 49.5)","Liquid Saturation well (49750) (1.4990E+04 10. 49.5)","Free I129 [M] well (49750) (1.4990E+04 10. 49.5)","Liquid Pressure [Pa] well (49751) (1.5010E+04 10. 49.5)","Liquid Saturation well (49751) (1.5010E+04 10. 49.5)","Free I129 [M] well (49751) (1.5010E+04 10. 49.5)","Liquid Pressure [Pa] well (50750) (1.4990E+04 10. 50.5)","Liquid Saturation well (50750) (1.4990E+04 10. 50.5)","Free I129 [M] well (50750) (1.4990E+04 10. 50.5)","Liquid Pressure [Pa] well (50751) (1.5010E+04 10. 50.5)","Liquid Saturation well (50751) (1.5010E+04 10. 50.5)","Free I129 [M] well (50751) (1.5010E+04 10. 50.5)","Liquid Pressure [Pa] well (51750) (1.4990E+04 10. 51.5)","Liquid Saturation well (51750) (1.4990E+04 10. 51.5)","Free I129 [M] well (51750) (1.4990E+04 10. 51.5)","Liquid Pressure [Pa] well (51751) (1.5010E+04 10. 51.5)","Liquid Saturation well (51751) (1.5010E+04 10. 51.5)","Free I129 [M] well (51751) (1.5010E+04 10. 51.5)","Liquid Pressure [Pa] well (52750) (1.4990E+04 10. 52.5)","Liquid Saturation well (52750) (1.4990E+04 10. 52.5)","Free I129 [M] well (52750) (1.4990E+04 10. 52.5)","Liquid Pressure [Pa] well (52751) (1.5010E+04 10. 52.5)","Liquid Saturation well (52751) (1.5010E+04 10. 52.5)","Free I129 [M] well (52751) (1.5010E+04 10. 52.5)","Liquid Pressure [Pa] well (53750) (1.4990E+04 10. 53.5)","Liquid Saturation well (53750) (1.4990E+04 10. 53.5)","Free I129 [M] well (53750) (1.4990E+04 10. 53.5)","Liquid Pressure [Pa] well (53751) (1.5010E+04 10. 53.5)","Liquid Saturation well (53751) (1.5010E+04 10. 53.5)","Free I129 [M] well (53751) (1.5010E+04 10. 53.5)","Liquid Pressure [Pa] well (54750) (1.4990E+04 10. 54.5)","Liquid Saturation well (54750) (1.4990E+04 10. 54.5)","Free I129 [M] well (54750) (1.4990E+04 10. 54.5)","Liquid Pressure [Pa] well (54751) (1.5010E+04 10. 54.5)","Liquid Saturation well (54751) (1.5010E+04 10. 54.5)","Free I129 [M] well (54751) (1.5010E+04 10. 54.5)","Liquid Pressure [Pa] well (55750) (1.4990E+04 10. 55.5)","Liquid Saturation well (55750) (1.4990E+04 10. 55.5)","Free I129 [M] well (55750) (1.4990E+04 10. 55.5)","Liquid Pressure [Pa] well (55751) (1.5010E+04 10. 55.5)","Liquid Saturation well (55751) (1.5010E+04 10. 55.5)","Free I129 [M] well (55751) (1.5010E+04 10. 55.5)","Liquid Pressure [Pa] well (56750) (1.4990E+04 10. 56.5)","Liquid Saturation well (56750) (1.4990E+04 10. 56.5)","Free I129 [M] well (56750) (1.4990E+04 10. 56.5)","Liquid Pressure [Pa] well (56751) (1.5010E+04 10. 56.5)","Liquid Saturation well (56751) (1.5010E+04 10. 56.5)","Free I129 [M] well (56751) (1.5010E+04 10. 56.5)","Liquid Pressure [Pa] well (57750) (1.4990E+04 10. 57.5)","Liquid Saturation well (57750) (1.4990E+04 10. 57.5)","Free I129 [M] well (57750) (1.4990E+04 10. 57.5)","Liquid Pressure [Pa] well (57751) (1.5010E+04 10. 57.5)","Liquid Saturation well (57751) (1.5010E+04 10. 57.5)","Free I129 [M] well (57751) (1.5010E+04 10. 57.5)","Liquid Pressure [Pa] well (58750) (1.4990E+04 10. 58.5)","Liquid Saturation well (58750) (1.4990E+04 10. 58.5)","Free I129 [M] well (58750) (1.4990E+04 10. 58.5)","Liquid Pressure [Pa] well (58751) (1.5010E+04 10. 58.5)","Liquid Saturation well (58751) (1.5010E+04 10. 58.5)","Free I129 [M] well (58751) (1.5010E+04 10. 58.5)","Liquid Pressure [Pa] well (59750) (1.4990E+04 10. 59.5)","Liquid Saturation well (59750) (1.4990E+04 10. 59.5)","Free I129 [M] well (59750) (1.4990E+04 10. 59.5)","Liquid Pressure [Pa] well (59751) (1.5010E+04 10. 59.5)","Liquid Saturation well (59751) (1.5010E+04 10. 59.5)","Free I129 [M] well (59751) (1.5010E+04 10. 59.5)","Liquid Pressure [Pa] well (60750) (1.4990E+04 10. 60.5)","Liquid Saturation well (60750) (1.4990E+04 10. 60.5)","Free I129 [M] well (60750) (1.4990E+04 10. 60.5)","Liquid Pressure [Pa] well (60751) (1.5010E+04 10. 60.5)","Liquid Saturation well (60751) (1.5010E+04 10. 60.5)","Free I129 [M] well (60751) (1.5010E+04 10. 60.5)","Liquid Pressure [Pa] well (61750) (1.4990E+04 10. 61.5)","Liquid Saturation well (61750) (1.4990E+04 10. 61.5)","Free I129 [M] well (61750) (1.4990E+04 10. 61.5)","Liquid Pressure [Pa] well (61751) (1.5010E+04 10. 61.5)","Liquid Saturation well (61751) (1.5010E+04 10. 61.5)","Free I129 [M] well (61751) (1.5010E+04 10. 61.5)","Liquid Pressure [Pa] well (62750) (1.4990E+04 10. 62.5)","Liquid Saturation well (62750) (1.4990E+04 10. 62.5)","Free I129 [M] well (62750) (1.4990E+04 10. 62.5)","Liquid Pressure [Pa] well (62751) (1.5010E+04 10. 62.5)","Liquid Saturation well (62751) (1.5010E+04 10. 62.5)","Free I129 [M] well (62751) (1.5010E+04 10. 62.5)","Liquid Pressure [Pa] well (63750) (1.4990E+04 10. 63.5)","Liquid Saturation well (63750) (1.4990E+04 10. 63.5)","Free I129 [M] well (63750) (1.4990E+04 10. 63.5)","Liquid Pressure [Pa] well (63751) (1.5010E+04 10. 63.5)","Liquid Saturation well (63751) (1.5010E+04 10. 63.5)","Free I129 [M] well (63751) (1.5010E+04 10. 63.5)","Liquid Pressure [Pa] well (64750) (1.4990E+04 10. 64.5)","Liquid Saturation well (64750) (1.4990E+04 10. 64.5)","Free I129 [M] well (64750) (1.4990E+04 10. 64.5)","Liquid Pressure [Pa] well (64751) (1.5010E+04 10. 64.5)","Liquid Saturation well (64751) (1.5010E+04 10. 64.5)","Free I129 [M] well (64751) (1.5010E+04 10. 64.5)","Liquid Pressure [Pa] well (65750) (1.4990E+04 10. 65.5)","Liquid Saturation well (65750) (1.4990E+04 10. 65.5)","Free I129 [M] well (65750) (1.4990E+04 10. 65.5)","Liquid Pressure [Pa] well (65751) (1.5010E+04 10. 65.5)","Liquid Saturation well (65751) (1.5010E+04 10. 65.5)","Free I129 [M] well (65751) (1.5010E+04 10. 65.5)","Liquid Pressure [Pa] well (66750) (1.4990E+04 10. 66.5)","Liquid Saturation well (66750) (1.4990E+04 10. 66.5)","Free I129 [M] well (66750) (1.4990E+04 10. 66.5)","Liquid Pressure [Pa] well (66751) (1.5010E+04 10. 66.5)","Liquid Saturation well (66751) (1.5010E+04 10. 66.5)","Free I129 [M] well (66751) (1.5010E+04 10. 66.5)","Liquid Pressure [Pa] well (67750) (1.4990E+04 10. 67.5)","Liquid Saturation well (67750) (1.4990E+04 10. 67.5)","Free I129 [M] well (67750) (1.4990E+04 10. 67.5)","Liquid Pressure [Pa] well (67751) (1.5010E+04 10. 67.5)","Liquid Saturation well (67751) (1.5010E+04 10. 67.5)","Free I129 [M] well (67751) (1.5010E+04 10. 67.5)","Liquid Pressure [Pa] well (68750) (1.4990E+04 10. 68.5)","Liquid Saturation well (68750) (1.4990E+04 10. 68.5)","Free I129 [M] well (68750) (1.4990E+04 10. 68.5)","Liquid Pressure [Pa] well (68751) (1.5010E+04 10. 68.5)","Liquid Saturation well (68751) (1.5010E+04 10. 68.5)","Free I129 [M] well (68751) (1.5010E+04 10. 68.5)","Liquid Pressure [Pa] well (69750) (1.4990E+04 10. 69.5)","Liquid Saturation well (69750) (1.4990E+04 10. 69.5)","Free I129 [M] well (69750) (1.4990E+04 10. 69.5)","Liquid Pressure [Pa] well (69751) (1.5010E+04 10. 69.5)","Liquid Saturation well (69751) (1.5010E+04 10. 69.5)","Free I129 [M] well (69751) (1.5010E+04 10. 69.5)","Liquid Pressure [Pa] well (70750) (1.4990E+04 10. 70.5)","Liquid Saturation well (70750) (1.4990E+04 10. 70.5)","Free I129 [M] well (70750) (1.4990E+04 10. 70.5)","Liquid Pressure [Pa] well (70751) (1.5010E+04 10. 70.5)","Liquid Saturation well (70751) (1.5010E+04 10. 70.5)","Free I129 [M] well (70751) (1.5010E+04 10. 70.5)","Liquid Pressure [Pa] well (71750) (1.4990E+04 10. 71.5)","Liquid Saturation well (71750) (1.4990E+04 10. 71.5)","Free I129 [M] well (71750) (1.4990E+04 10. 71.5)","Liquid Pressure [Pa] well (71751) (1.5010E+04 10. 71.5)","Liquid Saturation well (71751) (1.5010E+04 10. 71.5)","Free I129 [M] well (71751) (1.5010E+04 10. 71.5)","Liquid Pressure [Pa] well (72750) (1.4990E+04 10. 72.5)","Liquid Saturation well (72750) (1.4990E+04 10. 72.5)","Free I129 [M] well (72750) (1.4990E+04 10. 72.5)","Liquid Pressure [Pa] well (72751) (1.5010E+04 10. 72.5)","Liquid Saturation well (72751) (1.5010E+04 10. 72.5)","Free I129 [M] well (72751) (1.5010E+04 10. 72.5)","Liquid Pressure [Pa] well (73750) (1.4990E+04 10. 73.5)","Liquid Saturation well (73750) (1.4990E+04 10. 73.5)","Free I129 [M] well (73750) (1.4990E+04 10. 73.5)","Liquid Pressure [Pa] well (73751) (1.5010E+04 10. 73.5)","Liquid Saturation well (73751) (1.5010E+04 10. 73.5)","Free I129 [M] well (73751) (1.5010E+04 10. 73.5)","Liquid Pressure [Pa] well (74750) (1.4990E+04 10. 74.5)","Liquid Saturation well (74750) (1.4990E+04 10. 74.5)","Free I129 [M] well (74750) (1.4990E+04 10. 74.5)","Liquid Pressure [Pa] well (74751) (1.5010E+04 10. 74.5)","Liquid Saturation well (74751) (1.5010E+04 10. 74.5)","Free I129 [M] well (74751) (1.5010E+04 10. 74.5)","Liquid Pressure [Pa] well (75750) (1.4990E+04 10. 75.5)","Liquid Saturation well (75750) (1.4990E+04 10. 75.5)","Free I129 [M] well (75750) (1.4990E+04 10. 75.5)","Liquid Pressure [Pa] well (75751) (1.5010E+04 10. 75.5)","Liquid Saturation well (75751) (1.5010E+04 10. 75.5)","Free I129 [M] well (75751) (1.5010E+04 10. 75.5)","Liquid Pressure [Pa] well (76750) (1.4990E+04 10. 76.5)","Liquid Saturation well (76750) (1.4990E+04 10. 76.5)","Free I129 [M] well (76750) (1.4990E+04 10. 76.5)","Liquid Pressure [Pa] well (76751) (1.5010E+04 10. 76.5)","Liquid Saturation well (76751) (1.5010E+04 10. 76.5)","Free I129 [M] well (76751) (1.5010E+04 10. 76.5)","Liquid Pressure [Pa] well (77750) (1.4990E+04 10. 77.5)","Liquid Saturation well (77750) (1.4990E+04 10. 77.5)","Free I129 [M] well (77750) (1.4990E+04 10. 77.5)","Liquid Pressure [Pa] well (77751) (1.5010E+04 10. 77.5)","Liquid Saturation well (77751) (1.5010E+04 10. 77.5)","Free I129 [M] well (77751) (1.5010E+04 10. 77.5)","Liquid Pressure [Pa] well (78750) (1.4990E+04 10. 78.5)","Liquid Saturation well (78750) (1.4990E+04 10. 78.5)","Free I129 [M] well (78750) (1.4990E+04 10. 78.5)","Liquid Pressure [Pa] well (78751) (1.5010E+04 10. 78.5)","Liquid Saturation well (78751) (1.5010E+04 10. 78.5)","Free I129 [M] well (78751) (1.5010E+04 10. 78.5)","Liquid Pressure [Pa] well (79750) (1.4990E+04 10. 79.5)","Liquid Saturation well (79750) (1.4990E+04 10. 79.5)","Free I129 [M] well (79750) (1.4990E+04 10. 79.5)","Liquid Pressure [Pa] well (79751) (1.5010E+04 10. 79.5)","Liquid Saturation well (79751) (1.5010E+04 10. 79.5)","Free I129 [M] well (79751) (1.5010E+04 10. 79.5)"])
        time = result['Time [y]']
        idx  = [3*(i+1) for i in range(52)]
        cons = np.matrix(result)
        cons = cons[:,idx]
        well_cons = [np.mean(cons[i,:]) for i in range(len(time))]
        df[reactor] = well_cons 
    df.index        = time
    df = df.multiply(lam*6.02e23/3.9e10*1.0e12, axis='columns')
    
    return df

# Output Functions

In [10]:
def scaleGWey(workspace_path, st, I129_release):
    """
    Scaling factor to scale the results to per-1GWe.y equivalent
    """
    ## ---- Parameters: 
    # workspace_path (str) - path to R2R4SNF directory
    # st (int) - storage time for footprint minimization (as '#packages per GWe.y' change)
    # I129_release (df) - I129 release over time for all reactor types
    ## ---- Returns: 
    # I129_scaled (df) - release scaled to GWe.y

    rep_footprint = pd.read_csv(workspace_path + 'results/footprint_minimization/min_footprint_' + str(st) + '.csv')
    GWey_scale = dict(zip(rep_footprint['Reactor'], rep_footprint['#packages per GWe.y']))

    I129_scaled = I129_release.multiply(list(GWey_scale.values()), axis='columns')

    return I129_scaled

In [11]:
def scaleYM(reactors, parameters, I129_release):
    """
    Scaling factor to scale results to the YM repository of 70,000 MTHM
    """
    ## ---- Parameters: 
    # reactors (list) - reactor types
    # parameters (dict) - reactor parameter dictionary
    # I129_release (df) - I129 release over time for all reactor types
    ## ---- Returns: 
    # I129_scaled (df) - release scaled to YM repository maximum

    YM_scale = {}
    
    for reactor in reactors:
        # Calculate mass of heavy metal in the canister
        canister_loading = float(parameters[reactor]['hm_per_asm']) * float(parameters[reactor]['# fuel units']) 

        # Divide by YM repository size
        YM_scale[reactor] = 70000e6 / canister_loading

    I129_scaled = I129_release.multiply(list(YM_scale.values()), axis='columns')
    
    return I129_scaled

In [12]:
def write_output(workspace_path, reactors, parameters, level, f_matrix, I129_release, st = 50):
    """
    Save I129 release dataframes and remaining waste matrix under different scaling conditions
    """
    ## ---- Parameters:
    # workspace_path (str) - path to R2R4SNF directory
    # reactors (list) - reactor types
    # parameters (dict) - reactor parameter dictionary
    # level (str) - degradation rate (low or high)
    # f_matrix (df) - fraction of remaining waste matrix for all reactors
    # I129_release (df) - I129 release data for all reactors
    # st (int) - repository storage time for GWe.y calculations (default 50 years)
    
    # Set output path
    path = workspace_path + 'results/I129_release_transport/' + level + '_degradation/'
    
    # Write unscaled outputs
    f_matrix.to_csv(path + 'remaining_matrix.csv')
    I129_release.to_csv(path + 'I129_release.csv')

    # Write scaled I-129 YM output
    I129_YM_scaled = scaleYM(reactors, parameters, I129_release)
    I129_YM_scaled.to_csv(path + 'I129_release_YM.csv')
    
    # Write scaled I-129 GWE.y outputs
    I129_release_GWey = scaleGWey(workspace_path, st, I129_release)
    I129_release_GWey.to_csv(path + 'I129_release_GWey_' + str(st) + '.csv')

    # Save high-degradation release data as PFLOTRAN constraint
    if level == 'high':
        I129_release_rate4PFLOTRAN(workspace_path, I129_release_GWey)

In [13]:
def write_PFLOTRAN_output(workspace_path, reactors, parameters):
    """
    Save PFLOTRAN output to results file under different dosing conditions
    """

    ## ---- Parameters: 
    # workspace_path (str) - path to R2R4SNF directory    
    # reactors (list) - reactor types
    
    # Read PFLOTRAN results and save in results folder
    I129_conc = df_I129_PFLOTRAN2cons(workspace_path, reactors)
    I129_conc.to_csv(workspace_path + 'results/I129_release_transport/PFLOTRAN_output.csv')
    
    # Max concentration
    I129_conc_max = I129_conc.max().to_frame().T
    
    # Max dose rate
    bdcf = 3.36e-4 / 100 * 1e3 # biosphere dose conversion factor from YM (mSv/yr per pCi/L)
    I129_dose_max = I129_conc_max.multiply(bdcf)
    I129_dose_max.to_csv(workspace_path + 'results/I129_release_transport/I129_dose_max.csv', index = False)
    
    # GWey-equivalent dose rate
    bdcf_GWey =1.3e-4 #mSv/yr per pCi/L  https://www.nrc.gov/docs/ML0815/ML081560551.pdf
    I129_dose_max_GWey = I129_conc_max.multiply(bdcf_GWey)
    I129_conc_GWey = I129_conc.multiply(bdcf_GWey)
    
    I129_dose_max_GWey.to_csv(workspace_path + 'results/I129_release_transport/I129_dose_max_GWey.csv', index = False)
    I129_conc_GWey.to_csv(workspace_path + 'results/I129_release_transport/PFLOTRAN_output_GWey.csv')

    # YM-equivalent dose rate
    I129_dose_max_YM = scaleYM(reactors, parameters, I129_dose_max)
    I129_dose_max_YM.to_csv(workspace_path + 'results/I129_release_transport/I129_dose_max_YM.csv', index = False)

# Example

### Save I-129 Release Outputs

In [14]:
# Set up workspace
workspace_path = '/Users/krishnasunder/Desktop/workspace/R2R4SNF/'
reactors=['Ref_PWR', 'SPWR', 'HTGR', 'HPR', 'HTGR_FCM', 'SFR']

# Read in parameters
parameters = pd.read_csv(workspace_path + 'inputs/parameters/reactor_parameters.csv', index_col = 0).T.to_dict()

# Select the time series for the study of I129 release
time = pd.Series(np.concatenate([np.arange(10**i, 10**(i+1), 10**i) for i in range(7)]))

# Build dataframes and write output
for level in ('low', 'high'):
    f_matrix, I129_release = build_dfs(workspace_path, reactors, parameters, level, time)
    write_output(workspace_path, reactors, parameters, level, f_matrix, I129_release)

### PFLOTRAN Interface

PFLOTRAN needs to be run in each directory PFLOTRAN/pflotran_<reactor_name> before this next function. PFLOTRAN/run_pf.py can also be used to run multiple simulations at once

In [15]:
write_PFLOTRAN_output(workspace_path, reactors, parameters)